# Using Selenium, Beatutiful Soup & Requests: _Hershel, O My Bag & Whimsy and Row_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Hershel, O My Bag & Whimsy and Row. We will get the: picture,price, product link, and description.

Resources used to help in this process can be found in other notebooks.

#### Import your needed Libraries:

In [ ]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO
from scraping_functions import *
%load_ext autoreload
%autoreload 2

#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Hershel, O My Bag & Whimsy and Row_.

In [409]:
hershel_backpacks = 'https://herschel.com/shop/womens/backpacks'
hershel_fp = 'https://herschel.com/shop/womens/hip-packs'
hershel_totes = 'https://herschel.com/shop/womens/totes'
hershel_duffles = 'https://herschel.com/shop/womens/duffles'

O_my_bag = 'https://www.omybag.nl/product-category/women/women-bags/'

#only the first 13-ish bags
WR_bags = 'https://whimsyandrow.com/collections/bags-clutches-pouches?sort_by=title-ascending'

### O My Bag:

Now that we have our page variables set up, we can scrape the bags. After inspecting on the website it appears we can just use beautiful soup for this task.

In [ ]:
omb_page = requests.get(O_my_bag)
omb_soup = BeautifulSoup(omb_page.content, 'lxml')
divs_imgs = omb_soup.findAll('div', {'class':'content-product'})

In [ ]:
omb_links = omb_soup.findAll('a', {'class':'woocommerce-LoopProduct-link'})
omb_href = [x['href'] for x in omb_links]

Now let's grab the images:

In [ ]:
omb_images_tags = omb_soup.findAll('img', {'class':'attachment-woocommerce_thumbnail'})
omb_images = [x['src'] for x in omb_images_tags]

Now, let's get into the details like price and name:

In [ ]:
omb_prod_details = omb_soup.findAll('div', {'class':'product-details'})

In [ ]:
omb_names = [x.find('div', {'class': 'product-title'}).text.strip() for x in omb_prod_details]
omb_prices = [x.find('span', {'class': 'price'}).text.strip() for x in omb_prod_details]

### Create a DataFrame and use our function to download pictures :

Below we will use the lists created above and make a dataframe. Then we will define a function to download the images into our directory, and save the image file name in a new column of the DataFrame.

In [ ]:
table = [omb_names,omb_prices,omb_href,omb_images]
OMB_df = pd.DataFrame(table).transpose()

In [ ]:
OMB_df.rename( columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

In [ ]:
OMB_df.head(3)

#### Save all images using an apply

In [ ]:
c = 0
OMB_df['img'] = OMB_df.img_url.apply(lambda x: save_src_image_apply('../../data/','omybag_',x))

In [ ]:
OMB_df.to_csv('../../omb_df.csv')

### Whimsy and Row:

In [ ]:
wr_page = requests.get(WR_bags)
wr_soup = BeautifulSoup(wr_page.content, 'lxml')
divs_wr = wr_soup.findAll('div', {'class':'product-frame'})

In [ ]:
wr_url = ['https://whimsyandrow.com' + x.find('a')['href'] for x in divs_wr[:-3]]
wr_names = [x.find('a')['title'] for x in divs_wr[:-3]]
wr_images_url = ['https:'+x.find('img')['src'] for x in divs_wr[:-3]] 

Now, let's get the prices which are in a different div:

In [ ]:
divs_wr_2 = wr_soup.findAll('div', {'class':'product-title'})
prices = [x.find('h6').text for x in divs_wr_2[:-3]]

### Create this whimsy and Row DF and save the images:

In [ ]:
table = [wr_names,prices,wr_url,wr_images_url]
wr_df = pd.DataFrame(table).transpose()
wr_df.rename(columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

#### Save all images using an apply:

In [ ]:
c = 0
wr_df['img'] = wr_df.img_url.apply(lambda x: save_src_image_apply('../../data/','whimsy_row_',x))

In [ ]:
wr_df.to_csv('../../whimsy_row_df.csv')

### _Hershel_:

In [ ]:
h_page_1 = requests.get(hershel_backpacks)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
#three of the bookbags do not have multiple colors
colors_carousel = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]

h_bb_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel]

In [ ]:
h_bookbag_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_bb_imgs]
h_bookbag_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_bb_imgs]

In [ ]:
names = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]

In [ ]:
prices = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url = [x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

In [ ]:
names_adj = []
for i in range(len(names)):
    n = len(h_bb_imgs[i])
    names_adj.extend([names[i]]*n)

In [ ]:
prices_adj =  []
for i in range(len(prices)):
    n = len(h_bb_imgs[i])
    prices_adj.extend([prices[i]]*n)

In [ ]:
prod_h_url_adj =  []
for i in range(len(prod_h_url)):
    n = len(h_bb_imgs[i])
    prod_h_url_adj.extend([prod_h_url[i]]*n)

In [ ]:
len(h_bookbag_img),len(names)

In [ ]:
len(names_adj),len(prices_adj),len(prod_h_url_adj)

Let's flatten the lists of colors and images:

In [ ]:
h_bookbag_colors_fl = [color for sublist in h_bookbag_colors for color in sublist]
h_bookbag_img_fl = [img for sublist in h_bookbag_img for img in sublist]
# len(h_bookbag_colors_fl),len(h_bookbag_img_fl)

Make the DataFrame:

In [ ]:
table = [names_adj,prices_adj,h_bookbag_colors_fl,prod_h_url_adj,h_bookbag_img_fl]
herschel_df = pd.DataFrame(table).transpose()
herschel_df.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df['img'] = herschel_df.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel',x))

In [ ]:
herschel_df.to_csv('../../herschel_df.csv')

### Herschel Fannie Packs:

In [ ]:
h_page_1 = requests.get(hershel_fp)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
colors_carousel_fp = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_fp_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_fp]

In [ ]:
h_fannie_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_fp_imgs]
h_fannie_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_fp_imgs]
names_fp = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_fp = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_fp = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [ ]:
names_adj_fp = []
for i in range(len(names_fp)):
    n = len(h_bb_imgs_fp[i])
    names_adj_fp.extend([names_fp[i]]*n)
    
prices_adj_fp =  []
for i in range(len(prices_fp)):
    n = len(h_fp_imgs[i])
    prices_adj_fp.extend([prices_fp[i]]*n)

prod_h_url_adj_fp =  []
for i in range(len(prod_h_url_fp)):
    n = len(h_fp_imgs[i])
    prod_h_url_adj_fp.extend([prod_h_url_fp[i]]*n)

Flatten the images and colors lists:

In [ ]:
h_fp_colors_fl = [color for sublist in h_fannie_colors for color in sublist]
h_fp_img_fl = [img for sublist in h_fannie_img for img in sublist]
len(h_fp_colors_fl),len(h_fp_img_fl),len(prod_h_url_adj_fp)

Create the DataFrame of the Herschel Fannie Packs:

In [ ]:
table = [names_adj_fp,prices_adj_fp,h_fp_colors_fl,prod_h_url_adj_fp,h_fp_img_fl]
herschel_df_fp = pd.DataFrame(table).transpose()
herschel_df_fp.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [ ]:
herschel_df_fp.head()

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df_fp['img'] = herschel_df_fp.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_fp',x))

In [ ]:
herschel_df_fp.to_csv('../../herschel_df_fp.csv')

### Herschel Totes:

In [ ]:
h_page_1 = requests.get(hershel_totes)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
colors_carousel_t = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_t_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_t]

In [ ]:
h_tote_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_t_imgs]
h_tote_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_t_imgs]
names_t = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_t = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_t = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [ ]:
names_adj_t = []
for i in range(len(names_t)):
    n = len(h_t_imgs[i])
    names_adj_t.extend([names_t[i]]*n)
    
prices_adj_t =  []
for i in range(len(prices_t)):
    n = len(h_t_imgs[i])
    prices_adj_t.extend([prices_t[i]]*n)
1
prod_h_url_adj_t =  []
for i in range(len(prod_h_url_t)):
    n = len(h_t_imgs[i])
    prod_h_url_adj_t.extend([prod_h_url_t[i]]*n)

Flatten the images and colors lists:

In [ ]:
h_t_colors_fl = [color for sublist in h_tote_colors for color in sublist]
h_t_img_fl = [img for sublist in h_tote_img for img in sublist]
len(h_t_colors_fl),len(h_t_img_fl),len(prod_h_url_adj_t)

Create the DataFrame of the Herschel Fannie Packs:

In [ ]:
table = [names_adj_t,prices_adj_t,h_t_colors_fl,prod_h_url_adj_t,h_t_img_fl]
herschel_df_t = pd.DataFrame(table).transpose()
herschel_df_t.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [ ]:
herschel_df_t.head()

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df_t['img'] = herschel_df_t.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_t',x))

In [ ]:
herschel_df_t.to_csv('../../herschel_df_t.csv')

### Herschel Weekenders:


In [410]:
h_page_2 = requests.get(hershel_duffles)
h_soup_2 = BeautifulSoup(h_page_2.content, 'lxml')
divs_h = h_soup_2.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [411]:
colors_carousel_w = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_w_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_w]

In [412]:
h_wkndr_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_w_imgs]
h_wkndr_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_w_imgs]
names_w = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_w = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_w = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [413]:
names_adj_w = []
for i in range(len(names_w)):
    n = len(h_w_imgs[i])
    names_adj_w.extend([names_w[i]]*n)
    
prices_adj_w =  []
for i in range(len(prices_w)):
    n = len(h_w_imgs[i])
    prices_adj_w.extend([prices_w[i]]*n)
1
prod_h_url_adj_w =  []
for i in range(len(prod_h_url_w)):
    n = len(h_w_imgs[i])
    prod_h_url_adj_w.extend([prod_h_url_w[i]]*n)

Flatten the images and colors lists:

In [414]:
h_w_colors_fl = [color for sublist in h_wkndr_colors for color in sublist]
h_w_img_fl = [img for sublist in h_wkndr_img for img in sublist]
len(h_w_colors_fl),len(h_w_img_fl),len(prod_h_url_adj_w)

(219, 219, 219)

Create the DataFrame of the Herschel Fannie Packs:

In [415]:
table = [names_adj_w,prices_adj_w,h_w_colors_fl,prod_h_url_adj_w,h_w_img_fl]
herschel_df_w = pd.DataFrame(table).transpose()
herschel_df_w.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [416]:
herschel_df_w.head()

,product_name,product_price,color,URL,img_url
0,Novel Duffle,84.99,Black/Tan Synthetic Leather,https://herschel.com/shop/duffles/novel-duffle...,https://herschel.com/content/dam/herschel/prod...
1,Novel Duffle,84.99,Black/Black Synthetic Leather,https://herschel.com/shop/duffles/novel-duffle...,https://herschel.com/content/dam/herschel/prod...
2,Novel Duffle,84.99,Raven Crosshatch,https://herschel.com/shop/duffles/novel-duffle...,https://herschel.com/content/dam/herschel/prod...
3,Novel Duffle,84.99,Grey/Tan Synthetic Leather,https://herschel.com/shop/duffles/novel-duffle...,https://herschel.com/content/dam/herschel/prod...
4,Novel Duffle,84.99,Navy/Tan Synthetic Leather,https://herschel.com/shop/duffles/novel-duffle...,https://herschel.com/content/dam/herschel/prod...


#### Save all images using an apply:

In [417]:
c = 0
herschel_df_w['img'] = herschel_df_w.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_w',x))

Saved herschel_w_1.png
Saved herschel_w_2.png
Saved herschel_w_3.png
Saved herschel_w_4.png
Saved herschel_w_5.png
Saved herschel_w_6.png
Saved herschel_w_7.png
Saved herschel_w_8.png
Saved herschel_w_9.png
Saved herschel_w_10.png
Saved herschel_w_11.png
Saved herschel_w_12.png
Saved herschel_w_13.png
Saved herschel_w_14.png
Saved herschel_w_15.png
Saved herschel_w_16.png
Saved herschel_w_17.png
Saved herschel_w_18.png
Saved herschel_w_19.png
Saved herschel_w_20.png
Saved herschel_w_21.png
Saved herschel_w_22.png
Saved herschel_w_23.png
Saved herschel_w_24.png
Saved herschel_w_25.png
Saved herschel_w_26.png
Saved herschel_w_27.png
Saved herschel_w_28.png
Saved herschel_w_29.png
Saved herschel_w_30.png
Saved herschel_w_31.png
Saved herschel_w_32.png
Saved herschel_w_33.png
Saved herschel_w_34.png
Saved herschel_w_35.png
Saved herschel_w_36.png
Saved herschel_w_37.png
Saved herschel_w_38.png
Saved herschel_w_39.png
Saved herschel_w_40.png
Saved herschel_w_41.png
Saved herschel_w_42.png
S

In [418]:
herschel_df_w.to_csv('../../herschel_df_w.csv')

### Nordstrom Circle Bags:

In [465]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [466]:
base_url = 'https://shop.nordstrom.com/'
#create the chrome driver
start_time = time.time()
driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver') 
driver.get(base_url)

In [467]:
#now, search for the item we need
driver.find_element_by_class_name('ZlVcfv').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('ZaYgXf')
real_search.send_keys('circle bags')
time.sleep(1)
real_search.submit()
time.sleep(1)

In [468]:
#create a list of all of the products, and create and empty dictionary
bags = driver.find_elements_by_class_name('_2gErDB')

In [469]:
bags_2 = driver.find_elements_by_class_name('Z1AvvAr')

In [ ]:
img_urls = []
for i in bags_2:
    img_urls.append(i.get_attribute("href"))

In [498]:
#now, search for the item we need
driver.find_element_by_class_name('ZlVcfv').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('ZaYgXf')
real_search.send_keys('circle bags')
time.sleep(1)
real_search.submit()
time.sleep(1)

bags_2 = driver.find_elements_by_class_name('Z1AvvAr')
img_urls = []
for i in bags_2:
    img_urls.append(i.get_attribute("href"))

In [505]:
#len(bags_2)

In [ ]:
def get_attributes(image_l,price_l,name_l,color_l,url_l,brand_l):
    image_l.append(driver.find_element_by_class_name('_2g1r2h').get_attribute('src'))
    price_l.append(driver.find_element_by_class_name('Z1WEo3w').text)
    time.sleep(0.5)
    name_l.append(driver.find_element_by_class_name('_6VXnG').text)
    time.sleep(0.5)
    brand_l.append(driver.find_element_by_class_name('LGMSA').text)
    time.sleep(0.5)
    color_l.append(driver.find_element_by_class_name('_1QT7uJ').text)
    time.sleep(0.5)
    color_options = driver.find_elements_by_class_name('Z1MJbnq')
    url_l.append(driver.current_url)

In [ ]:
price_li = []
name_li = []
color = []
brand = []
url_li = []
img_li = []

for bag in img_urls:
    driver.get(bag)
    color_options = driver.find_elements_by_class_name('Z1MJbnq')
    if len(color_options) > 1:
        color_options[0].click()
        time.sleep(1)
        get_attributes(img_li,price_li,name_li,color,url_li,brand)
        for i in color_options[1:]:
            time.sleep(1)
            i.click()
            get_attributes(img_li,price_li,name_li,color,url_li,brand)
    else:
        time.sleep(1)
        get_attributes(img_li,price_li,name_li,color,url_li,brand)
        